In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as sts
from scipy.optimize import root

 ### Variables Setting

In [2]:
S = 100
K = 95
T = 0.5
div = 0
sigma = 0.2
r = 0.03
N = 5 ## Binomial Tree Node Number
Sim_num = 10000 ## Monte Carlo Simulation Number

 # Plain Vanila

 ## 1. Black Scholes Model

In [3]:
def BS_call(S, K, T, r, sigma, div = 0):
    d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = (S * np.exp(-div * T) * sts.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * sts.norm.cdf(d2, 0.0, 1.0))
    return call

def BS_put(S, K, T, r, sigma, div = 0):
    d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put = (K * np.exp(-r * T) * sts.norm.cdf(-d2, 0.0, 1.0) - S * np.exp(-div * T ) *sts.norm.cdf(-d1, 0.0, 1.0))
    return put

In [4]:
call = BS_call( S, K, T, r , sigma,div)
call

9.251090315022012

In [5]:
put = BS_put(S,K,T,r,sigma,div)
put

2.836724577312957

 ## 2. Monte Carlo Simulation

$$
\begin{array}{c}
 S_{T} = S_{0} e^{(r-\frac{\sigma^2}{2})T + \sigma \sqrt{T} \epsilon}
\end{array}
$$
\
$$
\begin{array}{c}
 C = e^{-rT} \hat{E}[ \ max(S_T - K , 0 )]
\end{array}
$$

In [6]:
def MC_call(S, K, T, r, sigma , div = 0, Sim_num = 10000) :
    e = np.random.normal(0,1,size = Sim_num)
    simul_S = S * np.exp((r-div-0.5*sigma**2)*T + sigma * np.sqrt(T) * e)
    Payoff = np.maximum(simul_S - K ,0)
    Discount_Payoff = np.exp(-r * T) * Payoff
    C = Discount_Payoff.mean()
    return C

def MC_put(S, K, T, r, sigma , div = 0, Sim_num = 10000) :
    e = np.random.normal(0,1,size = Sim_num)
    simul_S = S * np.exp((r-div-0.5*sigma**2)*T + sigma * np.sqrt(T) * e)
    Payoff = np.maximum( K-simul_S ,0)
    Discount_Payoff = np.exp(-r * T) * Payoff
    P = Discount_Payoff.mean()
    return P


In [7]:
call2 = MC_call(S , K, T, r , sigma, div, Sim_num)
call2

9.208747233919865

In [8]:
put2 = MC_put(S , K, T, r , sigma, div, Sim_num)
put2

2.911360404253014

 ## 3. Binomial Option Pricing

$$
\begin{array}{c}
 u = e^{\sigma \sqrt{\Delta t}} \ , \ d = \frac{1}{u} , \ p = \frac{e^{(r-\delta ) \Delta t} - d}{u-d}
\end{array}
$$


In [9]:
def UDP(r,div,sigma, T, N) :
    DeltaT = T/N
    u = np.exp(sigma * np.sqrt(DeltaT))
    d = 1/u
    p = (np.exp((r-div) * DeltaT) - d)/(u-d)
    return u, d, p

In [10]:
u,  d, p = UDP(r, div, sigma, T, N)

In [11]:
def S_Tree(S, N, u, d) :
    Temp = np.ones((N+1,N+1))
    Temp[:,0] = 0
    Temp1 = Temp.cumsum(1)
    Te = Temp.copy()
    Te[0,:] = 0
    num_u = np.triu(Temp1 - Te.cumsum(0))
    num_d = np.triu(Temp1 - num_u)
    Tree = (S * np.triu(u**num_u * d ** num_d))
    return Tree

 ### Make Underlying Tree

In [12]:
pd.DataFrame(S_Tree(S,N, u, d))

0           1           2           3           4           5
0  100.0  106.528839  113.483936  120.893120  128.786037  137.194270
1    0.0   93.871294  100.000000  106.528839  113.483936  120.893120
2    0.0    0.000000   88.118199   93.871294  100.000000  106.528839
3    0.0    0.000000    0.000000   82.717693   88.118199   93.871294
4    0.0    0.000000    0.000000    0.000000   77.648169   82.717693
5    0.0    0.000000    0.000000    0.000000    0.000000   72.889341

 ### Make Payoff Tree

In [13]:
Payoff_Tree = np.maximum(S_Tree(S,N,u,d) - K, 0)[:,-1:]
pd.DataFrame(Payoff_Tree, columns = ['Call_Payoff'])

Call_Payoff
0    42.194270
1    25.893120
2    11.528839
3     0.000000
4     0.000000
5     0.000000

In [14]:
def BN_call(S ,K ,T, r, sigma , div, N) :
    u,  d, p = UDP(r, div, sigma, T, N)
    Payoff_Tree = np.maximum(S_Tree(S,N,u,d) - K, 0)[:,-1:]
    Temp = np.ones((N+1,1))
    Temp[0] = 0
    Number_of_u = Temp.cumsum(0).astype(np.int64)[::-1]
    Weighted_Discount_Payoff = np.exp(-r*T) * ( Payoff_Tree * sts.binom.pmf(Number_of_u,N,p) )
    C = Weighted_Discount_Payoff.sum()    
    return C

def BN_put(S, K, T, r, sigma, div, N) :
    u,  d, p = UDP(r, div, sigma, T, N)
    Payoff_Tree = np.maximum( K - S_Tree(S,N,u,d), 0)[:,-1:]
    Temp = np.ones((N+1,1))
    Temp[0] = 0
    Number_of_u = Temp.cumsum(0).astype(np.int64)[::-1]
    Weighted_Discount_Payoff = np.exp(-r*T) * ( Payoff_Tree * sts.binom.pmf(Number_of_u,N,p) )
    P = Weighted_Discount_Payoff.sum()    
    return P

In [15]:
call3 = BN_call(S,K,T,r,sigma,div,N)
call3

9.186070147270566

In [16]:
put3 = BN_put(S,K,T,r,sigma,div,N)
put3

2.7717044095614995

 # Implied Volatility 

 ## 1. Black Scholes

In [17]:
def BS_call_IV(S,K,T,r,div,C) :
    def fun(x , S, K , T, r, div , P) :
        return BS_call(S,K,T,r,x,div)- P
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div, C))
    return x.x

def BS_put_IV(S,K,T,r,div,P) :
    def fun(x,S,K,T,r,div,Price) :
        return BS_put(S,K,T,r,x,div) - Price
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div, P))
    return x.x

In [18]:
BS_call_IV(S ,K,T,r,div, C = call)

array([0.2])

In [19]:
BS_put_IV(S ,K,T,r,div, P = put)

array([0.2])

 ## 2. Binomial

In [20]:
def BN_call_IV(S,K,T,r,div,C,N) :
    def fun(x , S, K , T, r, div , N,P) :
        return BN_call(S,K,T,r,x,div,N)- P
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div,N, C))
    return x.x

def BN_put_IV(S,K,T,r,div,P,N) :
    def fun(x,S,K,T,r,div,N,Price) :
        return BN_put(S,K,T,r,x,div,N) - Price
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div,N, P))
    return x.x

In [21]:
BN_call_IV(S ,K,T,r,div, C = call3, N = N)

array([0.2])

In [22]:
BN_put_IV(S ,K,T,r,div, P = put3, N = N)

array([0.2])

 # 연습문제
 
 ### 문제 1-1. 만기의 주가가 K 보다 큰 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 콜옵션의 Black Scholes 가격을 구하는 함수를 만드시오.
 
 ### 문제 1-2. 만기의 주가가 K 보다 작은 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 풋옵션의 Black Scholes 가격을 구하는 함수를 만드시오.

In [23]:
def BS_binary_call(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

def BS_binary_put(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

In [25]:
#bs_binary_c  = BS_binary_call(S,K,T,r,sigma,div) 
#bs_binary_c

0.6449574164253773

In [26]:
#bs_binary_p = BS_binary_put(S,K,T,r,sigma,div)
#bs_binary_p

0.3401545231776853

 ### 문제 2-1. 만기의 주가가 K 보다 큰 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 콜옵션의 이항모형 가격을 구하는 함수를 만드시오.
 
 ### 문제 2-2. 만기의 주가가 K 보다 작은 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 풋옵션의 이항모형 가격을 구하는 함수를 만드시오.

In [27]:
def BN_binary_call(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

def BN_binary_put(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

In [29]:
#bn_binary_call = BN_binary_call(S,K,T,r,sigma,div,N = 200)
#bn_binary_call

0.6416003497701392

In [30]:
#bn_binary_put = BN_binary_put(S,K,T,r,sigma,div,N = 200)
#bn_binary_put

0.34351158983292696

  ### 문제 3. 유러피안 배리어 콜옵션(down-and-out, B> K)의 closed form 가격은 다음과 같다. 이를 계산해주는 함수를 만드시오.
  \
$$
\begin{array}{c}
C_{down-and-out} = S e^{-\delta T} N(d_1 ) - K e^{-rT} N(d_2) - (\frac{B}{S})^{1+2(r-\delta)/\sigma^2} - S e^{-\delta T} N(h_1 ) + (\frac{B}{S})^{-1+2(r-\delta )/\sigma^2} Ke^{-rT} N(h_2 )
\\
h_1 = \frac{ ln(\frac{B^2}{KS})+(r-\delta + 0.5 \sigma^2)T }{\sigma \sqrt{T} }
\\
h_2 = h_1 - \sigma \sqrt{T}
\end{array}
$$  

In [31]:
def BS_call_down_and_out(S,K,T,r,sigma,div,Barrier = 0.9) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

In [33]:
#BS_call_down_and_out(S,K,T,r,sigma,div,Barrier = 0.97)

3.8243083263535738

 ### 문제 4. A 주식과 B 주식은 모두 100원에 거래되며 변동성은 각각 20%, 30% 이며, 수익률의 상관계수는 0.57이다. 만기에 다음과 같은 Payoff를 지급하는 옵션의 가치를 계산하는 함수를 만드시오. 변수는 S1, S2, K, T, r, sigma1, sigma2, corr,div1, div2, simul_num을 받으시오. \begin{array}{c} (S_A ,\ S_B 는 \ 각각 \ 만기의 \ A주식과 \ B주식의 \ 가치이다.) \end{array}
 
$$
\begin{array}{c}
Payoff = max(\sqrt{S_A\bullet S_B} - K , 0)
\end{array}
$$  

In [34]:
def customized_option(S1,S2,K,T,r,sigma1,sigma2,corr,div1,div2,Sim_num = 10000) :    
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################    
    return 

In [36]:
#customized_option(100,100,95,T,sigma1 = 0.2, sigma2 = 0.3, div1 = 0, div2 = 0, r = r , corr = 0.57)

9.63600532124204